# Join the poverty data, which is at the county level

In [1]:
import pandas as pd

In [2]:
unemployment_data = pd.read_excel('../Datasets/Unemployment.xlsx', skiprows=4) # first 4 rows are irrelevant

## The FIPS_code in the unemployment data is the 5-digit county fips code
It needs to be reformatted, as FIPS code with leading 0's were truncated

In [3]:
unemployment_data[['FIPS_code', 'State', 'Area_name']].head(5)

,FIPS_code,State,Area_name
0,0,US,United States
1,1000,AL,Alabama
2,1001,AL,"Autauga County, AL"
3,1003,AL,"Baldwin County, AL"
4,1005,AL,"Barbour County, AL"


In [4]:
# need to get rid of the state-level columns. 
unemployment_data.drop(0, inplace=True)
unemployment_data[['FIPS_code', 'State', 'Area_name']].head(5)

,FIPS_code,State,Area_name
1,1000,AL,Alabama
2,1001,AL,"Autauga County, AL"
3,1003,AL,"Baldwin County, AL"
4,1005,AL,"Barbour County, AL"
5,1007,AL,"Bibb County, AL"


# Reformat FIPS_code and rename as "CountyFIPS"

In [5]:
unemployment_data['FIPS_code'] = unemployment_data['FIPS_code'].astype(str).str.pad(width=5, side='left', fillchar='0')

In [6]:
unemployment_data[['FIPS_code', 'State', 'Area_name']].head(5)

,FIPS_code,State,Area_name
1,01000,AL,Alabama
2,01001,AL,"Autauga County, AL"
3,01003,AL,"Baldwin County, AL"
4,01005,AL,"Barbour County, AL"
5,01007,AL,"Bibb County, AL"


In [7]:
unemployment_data.rename(columns={'FIPS_code': 'CountyFIPS'}, inplace=True)

# Check the county overlap with the training data

In [8]:
# need to make sure these columns get read as strings
fips_columns = [
    'CountyFIPS',
    'CityFIPS',
    'TractFIPS'
]
dtypes_map = {x: 'str' for x in fips_columns}
training_data = pd.read_csv('../Datasets/joined_health_data.csv', dtype=dtypes_map)

In [9]:
unemployment_data_counties = set(unemployment_data['CountyFIPS'])
training_data_counties = set(training_data['CountyFIPS'])

In [10]:
assert len(training_data_counties.intersection(unemployment_data_counties)) ==\
        len(training_data_counties)
print('All of the counties in the training data are accounted for in the unemployment data')

All of the counties in the training data are accounted for in the unemployment data


# Engineer some columns into the unemployment dataset, and then join to the training dataset

In [11]:
# only keep these columns
unemployment_data = unemployment_data[[
    'CountyFIPS',
    'Unemployment_rate_2016',
    'Unemployment_rate_2017'
]]

In [12]:
unemployment_data['Unemployment_trend'] =\
    100 * (unemployment_data['Unemployment_rate_2017'] - unemployment_data['Unemployment_rate_2016']) / unemployment_data['Unemployment_rate_2016']

In [13]:
unemployment_data = unemployment_data.set_index('CountyFIPS') # set as index for join
unemployment_data = unemployment_data.drop(columns=['Unemployment_rate_2016'])
unemployment_data.head(3)

,Unemployment_rate_2017,Unemployment_trend
CountyFIPS,,
01000,4.5,-23.728814
01001,4.0,-21.568627
01003,4.2,-22.222222


In [14]:
training_data.shape[0]

27119

In [15]:
training_data = training_data.join(unemployment_data, on='CountyFIPS') 

In [16]:
new_column_names = unemployment_data.columns
for new_column_name in new_column_names:
    col = training_data.pop(new_column_name)
    training_data.insert(6, new_column_name, col)

In [17]:
training_data.head(3)

,StateAbbr,StateDesc,CityName,CountyFIPS,TractFIPS,OBESITY_2017,Unemployment_trend,Unemployment_rate_2017,OBESITY_trend,CHECKUP_2017,...,BPMED_trend,CANCER_2017,CANCER_trend,BINGE_2017,BINGE_trend,HIGHCHOL_2017,HIGHCHOL_trend,DIABETES_2017,DIABETES_trend,DIABETES_3Y_Change_Percentage
0,AL,Alabama,Birmingham,01073,01073000100,46.4,-22.807018,4.4,-1.902748,74.8,...,-2.750000,5.3,1.923077,10.3,-10.434783,35.3,-12.623762,17.4,-4.918033,9.770115
1,AL,Alabama,Birmingham,01073,01073000300,47.8,-22.807018,4.4,0.631579,72.1,...,-2.241594,4.7,2.173913,10.5,-12.500000,36.8,-11.325301,19.8,0.507614,10.101010
2,AL,Alabama,Birmingham,01073,01073000400,45.7,-22.807018,4.4,-3.177966,76.6,...,-3.170732,5.3,1.923077,10.4,-10.344828,35.6,-15.839243,18.4,-6.122449,2.717391


In [18]:
training_data.to_csv('../Datasets/training_data_full.csv', index=False)